# Bosques Aleatorios

Hasta el momento, hemos visto que el **Machine Learning** se aplica de *tres maneras* diferentes:

- El Aprendizaje Supervisado
- El Aprendizaje no Supervisado
- El Aprendizaje por Reforzamiento

El **Aprendizaje Supervisado** (que es un modelo que puede hacer predicciones sobre nuevos datos en base a lo aprendido en el entrenamiento con datos etiquetados), se puede aplicar usando *4 algoritmos* distintos:

* Regresión Lineal
* Regresión Logística
* Árboles de Decisión
* Bosques Aleatorios

En las lecciones anteriores hemos aprendido a hacer **Regresión Lineal**, **Regresión Logística**, y **Árboles de Decisión**, por lo que nos falta ver el último algoritmo propuesto, que es el **Bosque Aleatorio**.

Al igual que el *Árbol de Decisión*, el **Bosque aleatorio** puede usarse tanto para *regresión* como para *clasificación*. Es un método que básicamente lo que hace es **construir múltiples árboles de decisión** y combina sus salidas para hacer una predicción final.

Los bosques aleatorios tienen **varias ventajas** sobre los árboles de decisión individuales. Primero, que son menos propensos a sobreajustarse, y segundo que pueden manejar un mayor número de características de entrada. Pero creo que la mayor ventaja es que son capaces de capturar relaciones no lineales complejas entre las variables de entrada y la salida.

Vamos a probar este modelo con un conjunto de datos bien complejo.

Primero vamos a importar a las librerías que ya conocemos, y a otras que vamos a conocer en esta lección.

In [1]:
# Librerías conocidas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Librerías nuevas
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

En segundo lugar vamos a cargar nuestros datos en un `df`. Recuerda descargar este archivo **csv** desde los recursos de esta lección.

In [2]:
ruta = "C:/Users/Federico/Downloads/Python para Data Science/Día 11/7 - Bosque Aleatorio/tarjetas_credito.csv"

df = pd.read_csv(ruta)
df.head()

,Duracion,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Monto,Clase
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Esta es una base de datos que contiene casi *285 mil registros* de transacciones hechas con **tarjetas de crédito**. Sí que es un gran dataset. 

Observa que los nombres de las columnas son bastante crípticos, porque no nos permiten saber de qué se trata, pero esto es porque cuando se trabaja con información sensible o privada, normalmente su contenido pasa por algún proceso de anonimización, para poder manipular esos datos sin tener acceso a información restringida. Por eso los nombres de la mayoría de las columnas son `Valor1`, `Valor2`, etc. Es muy posible que cada columna represente datos como *ubicación*, *hora y fecha*, *tipo de comercio*, *historial de fraudes*, etc. Pero con todos sus valores codificados en números para respetar la integridad de la información.

Las columnas que sí tenemos explicadas son `Duración`, `Monto` y `Clase`.

La columna `Clase` es la más importante, porque es la que dice si la transacción fue legítima (con el valor **0**) o fraudulenta (con el valor **1**). Entonces el objetivo de nuestro trabajo va a ser usar un algoritmo de **bosque aleatorio** para identificar si hay alguna relación fuerte entre todas las columnas del registro y la columna `Clase`, de modo que luego podamos usar este modelo para identificar si una nueva transacción tiene riesgo de ser fraudulenta o no.

El primer paso, en este caso va a ser **normalizar los datos**. ¿Qué es eso? Cuando tenemos toda esta cantidad de números tan diversos, ya que cada columna representa diferentes cosas, y sus datos pueden tener valores mínimos y máximos muy dispersos, va a ser muy difícil para nuestro modelo encontrar relaciones entre todos ellos. Entonces vamos a tener que normalizar a estos datos.

Normalizar los datos es hacer que todos estos números de todas estas columnas sean **transformados dentro de una nueva escala**, para que en cada columna el *valor mínimo* sea **0** y el *valor máximo* sea **1**. Esto va a ayudar al algoritmo a entender las relaciones que pueden llegar a haber entre ellos.

Aquí es donde entra en acción la primera librería nueva que vamos a conocer en esta lección:

In [3]:
escala = MinMaxScaler(feature_range=(0, 1))
normado = escala.fit_transform(df)
df_normado = pd.DataFrame(data=normado, columns=df.columns)
df_normado.head()

,Duracion,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Monto,Clase
0,0.000000,0.935192,0.766490,0.881365,0.313023,0.763439,0.267669,0.266815,0.786444,0.475312,...,0.561184,0.522992,0.663793,0.391253,0.585122,0.394557,0.418976,0.312697,0.005824,0.0
1,0.000000,0.978542,0.770067,0.840298,0.271796,0.766120,0.262192,0.264875,0.786298,0.453981,...,0.557840,0.480237,0.666938,0.336440,0.587290,0.446013,0.416345,0.313423,0.000105,0.0
2,0.000006,0.935217,0.753118,0.868141,0.268766,0.762329,0.281122,0.270177,0.788042,0.410603,...,0.565477,0.546030,0.678939,0.289354,0.559515,0.402727,0.415489,0.311911,0.014739,0.0
3,0.000006,0.941878,0.765304,0.868484,0.213661,0.765647,0.275559,0.266803,0.789434,0.414999,...,0.559734,0.510277,0.662607,0.223826,0.614245,0.389197,0.417669,0.314371,0.004807,0.0
4,0.000012,0.938617,0.776520,0.864251,0.269796,0.762975,0.263984,0.268968,0.782484,0.490950,...,0.561327,0.547271,0.663392,0.401270,0.566343,0.507497,0.420561,0.317490,0.002724,0.0


Ahora tenemos un **nuevo DataFrame** cuya única diferencia con el original es que ahora todos sus datos se encuentran dentro del rango que va de **0** a **1**, pero manteniendo entre ellos, por supuesto, la misma estructura y la misma distancia.

Ahora sí, comencemos a definir nuestras variables. La **variable independiente** va a ser un DataFrame que contenga a **todas las columnas menos `Clase`**, que es la que queremos averiguar.

In [5]:
X = df_normado.drop("Clase", axis=1)

Y la **variable dependiente**, que es la que queremos descubrir, va a ser justamente la columna `Clase`:

In [6]:
y = df_normado["Clase"]

Ahora creamos nuestros grupos de **entrenamiento** y **prueba** como lo hemos hecho hasta ahora:

In [8]:
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(X, y, train_size=0.7, random_state=42)

Y llamamos al **algoritmo** de **bosque aleatorio** (que ya hemos importado y que se llama `RandomForestClassifier()`) para poder entrenarlo.

Esto va a tomar **bastante tiempo**, porque es un algoritmo complejo y porque es un dataset enorme, por lo que no te preocupes si ves que te toma muchos minutos.

In [9]:
forest = RandomForestClassifier()
forest.fit(X_entrena, y_entrena)

RandomForestClassifier()

Ahora que el modelo está entrenado, voy a aplicar el método score para ver qué tan confiable es mi modelo para establecer **predicciones** en base a los datos de este dataset.

In [10]:
forest.score(X_prueba, y_prueba)

0.9996254813150287

Como puedes ver el puntaje es bastante alto, y esto quiere decir que este es un modelo muy **apropiado para identificar si una futura transacción puede ser fraudulenta o no**, ya que ha encontrado **relaciones muy fuertes** entre todas las variables independientes y la salida de la columna `Clase`.

Ahora simulemos una situación en la que recibimos una **nueva transacción**, de la cual queremos saber qué tan probable es que sea fraudulenta, y para eso vamos a aplicar este modelo que parece ser tan confiable.

Voy a crear un nuevo registro con este código:

In [11]:
nuevo_registro = pd.DataFrame({
    'Duracion': [0.000006], 'V1': [0.452345], 'V2': [0.564789], 'V3': [0.123456], 'V4': [0.654321],
    'V5': [0.987654], 'V6': [0.345678], 'V7': [0.234567], 'V8': [0.876543], 'V9': [0.456789],
    'V10': [0.567890], 'V11': [0.678901], 'V12': [0.789012], 'V13': [0.890123], 'V14': [0.901234],
    'V15': [0.012345], 'V16': [0.543210], 'V17': [0.432109], 'V18': [0.321098], 'V19': [0.210987],
    'V20': [0.109876], 'V21': [0.098765], 'V22': [0.887654], 'V23': [0.776543], 'V24': [0.665432],
    'V25': [0.554321],     'V26': [0.443210], 'V27': [0.332109], 'V28': [0.221098], 'Monto': [0.110987]
}, index=[0])

Ahora voy a usar el método `predict()` de `RandomForestClasifier()` para hacer rápidamente mi predicción de si va a ser clase **0** o **1**.

In [12]:
clase_predicha = forest.predict(nuevo_registro)
clase_predicha

array([0.])

Esas son buenas noticias, parece que esta nueva operación **es legítima**.

Ahora usemos `predict_proba()` para ver qué tan probable es que sea legítima.

In [13]:
probabilidades = forest.predict_proba(nuevo_registro)
probabilidades

array([[0.62, 0.38]])

Bueno, parece que hay algunos indicadores de duda, ya que hay solo un **62% de posibilidades de que sea legítima**, pero el resultado sigue siendo positivo. Pasó la prueba.

Y ahora hagamos un pequeño bloque de código para mostrar esta información de un modo más amigable.

In [15]:
print("Clase predicha: ", clase_predicha[0])
print("Probabilidades de Legitimidad: ", probabilidades[0][0])
print("Probabilidades de Fraude: ", probabilidades[0][1])

Clase predicha:  0.0
Probabilidades de Legitimidad:  0.62
Probabilidades de Fraude:  0.38


Este ha sido el algoritmo de **bosque aleatorio** que hemos usado para que la máquina haga **aprendizaje supervisado**.